In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from typing import Iterable
import envs
import pandas as pd
import string

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

/home/rogger/miniconda3/envs/DoLa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.28.1


In [2]:
import DoLa.dola as dola

In [3]:
model = dola.DoLa(MODEL_NAME, "cuda", num_gpus=1)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 12.48it/s]


In [4]:
input_text = "Where is the capital of Washington State?"
layers = [16,18,20,22,24,26,28,30,32]

In [5]:
model.generate(input_text,
               temperature=0.7,
               mature_layer=layers[-1],
               candidate_premature_layers=layers[:-1],
               mode="dola")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MODEL OUTPUT: 


The capital city of Washington State is Olympia.


('\n\nThe capital city of Washington State is Olympia.',
 {16: 12, 18: 0, 20: 1, 22: 0, 24: 0, 26: 0, 28: 0, 30: 0})

In [6]:
def gen_func(source, debug=False):
    input_text = "<s> [INST] " + envs.SYSTEM_PROMPT + " \n\n" + envs.USER_PROMPT.format(passage=source) + " [/INST]"
    if debug: print(input_text)
    out = model.generate(input_text,
                         do_sample=False,
                         max_new_tokens=512,
                         pad_token_id=model.tokenizer.eos_token_id,
                         mature_layer=layers[-1],
                         candidate_premature_layers=layers[:-1],
                         mode="dola",
                         verbose=False)
    return out[0]

In [7]:
text = gen_func("The first vaccine for Ebola was approved by the FDA in 2019 in the US, five years after the initial outbreak in 2014. To produce the vaccine, scientists had to sequence the DNA of Ebola, then identify possible vaccines, and finally show successful clinical trials. Scientists say a vaccine for COVID-19 is unlikely to be ready this year, although clinical trials have already started.", debug=True)
print(text)

<s>[INST] You are a chat bot answering questions using data. You must stick to the answers provided solely by the text in the passage provided. 

You are asked the question 'Provide a concise summary of the following passage, covering the core pieces of information described': 
Passage:
The first vaccine for Ebola was approved by the FDA in 2019 in the US, five years after the initial outbreak in 2014. To produce the vaccine, scientists had to sequence the DNA of Ebola, then identify possible vaccines, and finally show successful clinical trials. Scientists say a vaccine for COVID-19 is unlikely to be ready this year, although clinical trials have already started. [/INST]
The first Ebola vaccine was approved by the FDA in the US in 2019, five years after the initial outbreak in 2014. Scientists had to sequence the DNA of Ebola, identify possible vaccines, and conduct successful clinical trials to produce the vaccine. A vaccine for COVID-19 is unlikely to be ready this year, despite cli

In [8]:
from leaderboard import SummaryGenerator, EvaluationModel
summ = SummaryGenerator()

In [9]:
df = summ.generate_summaries(pd.read_csv("leaderboard_dataset.csv"), gen_func)
df.to_csv("generated.csv", index=False)

  0%|          | 0/1006 [00:00<?, ?it/s]

100%|██████████| 1006/1006 [1:24:16<00:00,  5.03s/it]
